In [1]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.6MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.6 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=85dc76d1e2d881af716a9edb1d4c8ac7c40ae6102247c5270d1b8bae7e7b327b
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [537]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle -v

Kaggle API 1.5.6


In [482]:

!kaggle competitions download -c train-person

train-person.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:

!unzip train-person.zip

Archive:  train-person.zip
  inflating: new_test.csv            
  inflating: new_train.csv           
  inflating: submit_sample.csv       


In [552]:
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

drop_prob = 0.5

In [553]:
device = 'cuda'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [554]:
train=pd.read_csv("new_train.csv")
x_train=train.iloc[:,1:]
y_train=train.iloc[:,[0]]

x_train=np.array(x_train)
y_train=np.array(y_train)

x_train=torch.FloatTensor(x_train)
y_train=torch.LongTensor(y_train)

In [555]:
train

,index,08~09,09~10,10~11,11~12,12~13
0,1,127,163,13,129,199
1,0,63,78,72,95,118
2,1,56,92,53,29,91
3,0,27,26,35,37,3
4,1,107,131,133,14,152
...,...,...,...,...,...,...
5995,0,52,73,56,69,104
5996,1,215,233,204,289,254
5997,0,165,241,259,30,36
5998,1,167,162,148,168,248


In [556]:
y_train

tensor([[1],
        [0],
        [1],
        ...,
        [0],
        [1],
        [0]])

In [557]:
test=pd.read_csv("new_test.csv")
test

,08~09,09~10,10~11,11~12,12~13
0,251,260,268,268,313
1,180,207,233,291,283
2,120,127,149,200,190
3,194,299,234,245,261
4,136,153,200,265,334
...,...,...,...,...,...
683,60,88,95,92,103
684,49,47,61,45,68
685,27,39,45,32,42
686,148,207,209,179,224


In [558]:
y_train=y_train.squeeze()
y_train.shape

torch.Size([6000])

In [559]:
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
data_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=32,
                                          shuffle=True,
                                          drop_last=True)

In [560]:
l1 = torch.nn.Linear(5, 130)
l2 = torch.nn.Linear(130,130)
l3 = torch.nn.Linear(130,130)
l5 = torch.nn.Linear(130,130)
l6 = torch.nn.Linear(130,130)
l4 = torch.nn.Linear(130,2)

relu = torch.nn.ReLU()

torch.nn.init.xavier_uniform_(l1.weight)
torch.nn.init.xavier_uniform_(l2.weight)
torch.nn.init.xavier_uniform_(l3.weight)
torch.nn.init.xavier_uniform_(l4.weight)
torch.nn.init.xavier_uniform_(l5.weight)
torch.nn.init.xavier_uniform_(l6.weight)

bn1=torch.nn.BatchNorm1d(130)
bn2=torch.nn.BatchNorm1d(130)
bn3=torch.nn.BatchNorm1d(130)
bn4=torch.nn.BatchNorm1d(130)
bn5=torch.nn.BatchNorm1d(130)


model = torch.nn.Sequential(l1,bn1, relu,
                            l2,bn2,relu,
                            l3,bn3,relu,
#                            l5,bn4,relu,
#                            l6,bn5,relu,
                            l4).to(device)

                            
model

Sequential(
  (0): Linear(in_features=5, out_features=130, bias=True)
  (1): BatchNorm1d(130, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=130, out_features=130, bias=True)
  (4): BatchNorm1d(130, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Linear(in_features=130, out_features=130, bias=True)
  (7): BatchNorm1d(130, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Linear(in_features=130, out_features=2, bias=True)
)

In [561]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) 

In [562]:
total_batch = len(data_loader)
model_h = []
error_h = []
for epoch in range(1,300):
  avg_cost = 0

  for X, Y in data_loader:

        X = X.to(device)
        Y = Y.to(device)

      
  optimizer.zero_grad()
  hypothesis = model(X)
        
  cost = loss(hypothesis, Y)
  cost.backward()
  optimizer.step()
  avg_cost += cost
  avg_cost /= total_batch
  model_h.append(model)
  error_h.append(avg_cost)

  if epoch % 10 == 0 :
        print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

print('Epoch:', '%04d' % (epoch), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0010 cost = 0.004717848
Epoch: 0020 cost = 0.003453984
Epoch: 0030 cost = 0.003639182
Epoch: 0040 cost = 0.003549270
Epoch: 0050 cost = 0.003838530
Epoch: 0060 cost = 0.002992020
Epoch: 0070 cost = 0.003837575
Epoch: 0080 cost = 0.003223554
Epoch: 0090 cost = 0.003385795
Epoch: 0100 cost = 0.003199906
Epoch: 0110 cost = 0.002732291
Epoch: 0120 cost = 0.003372659
Epoch: 0130 cost = 0.003633119
Epoch: 0140 cost = 0.004158460
Epoch: 0150 cost = 0.003701375
Epoch: 0160 cost = 0.003695688
Epoch: 0170 cost = 0.004234017
Epoch: 0180 cost = 0.003830374
Epoch: 0190 cost = 0.003999668
Epoch: 0200 cost = 0.003279037
Epoch: 0210 cost = 0.003476872
Epoch: 0220 cost = 0.002941837
Epoch: 0230 cost = 0.003714714
Epoch: 0240 cost = 0.003906392
Epoch: 0250 cost = 0.003328238
Epoch: 0260 cost = 0.003854216
Epoch: 0270 cost = 0.003166377
Epoch: 0280 cost = 0.003067336
Epoch: 0290 cost = 0.003732142
Epoch: 0299 cost = 0.003265693


In [563]:
best_model = model_h[np.argmin(error_h)]
test=np.array(test)

test=torch.FloatTensor(test)
test = test.to(device)

with torch.no_grad():

    pred = best_model(test)
    predict=torch.argmax(pred,dim=1)

    print(predict.shape)

torch.Size([688])


In [564]:
submit = pd.read_csv('submit_sample.csv')

predict=predict.cpu().numpy().reshape(-1,1)

id=np.array([i for i in range(len(predict))]).reshape(-1,1)
result=np.hstack([id,predict])

submit=pd.DataFrame(result,columns=["id","index"])
submit.to_csv("submission.csv",index=False,header=True)

In [565]:
!kaggle competitions submit -c train-person -f submission.csv -m "4layer 48(1) -> 128"

100% 3.93k/3.93k [00:06<00:00, 630B/s]
Successfully submitted to  SejongAI.텀프로젝트.[시간대별 승하차 여부 예측하기] 